In [1]:
# Standard library imports
import warnings
from pathlib import Path

# Third-party imports
import pandas as pd

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

from settings import config
from pull_taq import get_taq_nbbo, get_taq_wct
from transform_taq import extract_features

In [2]:
# Change default pandas display options

pd.options.display.max_columns = 30
pd.options.display.max_colwidth = 200
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.expand_frame_repr', False)

# Global variables
RAW_DATA_DIR = Path(config("RAW_DATA_DIR"))
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WRDS_USERNAME = config("WRDS_USERNAME")

## 1. Load TAQ data

### 1.1. Quotes Data

Pull quotes data from TAQ database (NBBO)
- Display first 5 rows to confirm data is loaded correctly

In [3]:
quotes = get_taq_nbbo(('SPY'), date='2024-03-07', use_bars=False)
display(quotes.head())

,ticker,date,time_m,best_bid,best_bidsizeshares,best_ask,best_asksizeshares,time_quote
0,SPY,2024-03-07,0 days 09:30:00.000077,513.1100,5600,513.1400,400,2024-03-07 09:30:00.000077056-05:00
1,SPY,2024-03-07,0 days 09:30:00.000834,513.1100,5600,513.1400,500,2024-03-07 09:30:00.000834792-05:00
2,SPY,2024-03-07,0 days 09:30:00.000999,513.1100,3700,513.1400,500,2024-03-07 09:30:00.000999168-05:00
3,SPY,2024-03-07,0 days 09:30:00.001110,513.1100,3800,513.1400,500,2024-03-07 09:30:00.001110504-05:00
4,SPY,2024-03-07,0 days 09:30:00.001661,513.1100,3800,513.1300,100,2024-03-07 09:30:00.001661184-05:00


### 1.2. Trades Data

Pull quotes data from Trades database (WCT)
- Display first 5 rows to confirm data is loaded correctly

In [4]:
trades = get_taq_wct(('SPY'), date='2024-03-07')
display(trades.head())

,ticker,date,time_trade,price,size
0,SPY,2024-03-07,2024-03-07 09:30:00.000998-05:00,513.1100,1900
1,SPY,2024-03-07,2024-03-07 09:30:00.015429-05:00,513.1400,100
2,SPY,2024-03-07,2024-03-07 09:30:00.015486-05:00,513.1400,100
3,SPY,2024-03-07,2024-03-07 09:30:00.027825-05:00,513.1300,100
4,SPY,2024-03-07,2024-03-07 09:30:00.028134-05:00,513.1400,100


In [5]:
trades.dtypes

ticker                          string[python]
date                                    object
time_trade    datetime64[ns, America/New_York]
price                                  Float64
size                                     Int64
dtype: object

## 2. Data Preparation & feature Engineering

#### 2.1. Merge Quotes and Trades

Asof-join of quotes and trades dataframes
- Match each trade with the most recent quote, keeping only rows with actual trades (left join) (that naturally discards pure-quote timestamps that lack trades.)
- Display first 5 rows to confirm data is loaded correctly

In [6]:
# Asof-join on trade_ts (left side) to quote_ts (right side).
merged_trades = pd.merge_asof(
    trades.sort_values("time_trade"),
    quotes.sort_values("time_quote")[["time_quote", "best_bid", "best_bidsizeshares", "best_ask", "best_asksizeshares"]],
    left_on="time_trade",
    right_on="time_quote",
    direction="backward"  # Ensures we take the most recent quote before the trade
)
merged_trades = merged_trades.drop(columns="time_quote")
display(merged_trades.head())


,ticker,date,time_trade,price,size,best_bid,best_bidsizeshares,best_ask,best_asksizeshares
0,SPY,2024-03-07,2024-03-07 09:30:00.000998-05:00,513.1100,1900,513.1100,5600,513.1400,500
1,SPY,2024-03-07,2024-03-07 09:30:00.015429-05:00,513.1400,100,513.1200,1600,513.1300,100
2,SPY,2024-03-07,2024-03-07 09:30:00.015486-05:00,513.1400,100,513.1200,1600,513.1300,100
3,SPY,2024-03-07,2024-03-07 09:30:00.027825-05:00,513.1300,100,513.1200,3300,513.1400,200
4,SPY,2024-03-07,2024-03-07 09:30:00.028134-05:00,513.1400,100,513.1300,2400,513.1400,200


### 2.2. Feature Engineering

#### Extracted features:

**Duration since last trade**:
- Time since the last trade.

**Mid price**:
- Average of the best bid and ask prices.
$$MP = \frac{\text{best bid price} + \text{best ask price}}{2}$$


**EWMA Price Returns**:
- To mitigate tick sensitivity, we can compute returns using an exponentially weighted moving average (EWMA) of past prices.
$$
r_t^{(T)} = \log \left( \frac{P_t}{\text{EWMA} \left(P, \lambda = \frac{1}{T} \right)} \right)
$$
- This smooths out price fluctuations and provides a more stable return measure.



**Order-weighted average price**:
- Weighted average of the best bid and ask prices.
$$
OWA = \frac{\sqrt{\text{ask size}}}{\sqrt{\text{ask size} + \text{bid size}}} \times \text{best bid price} + \frac{\sqrt{\text{bid size}}}{\sqrt{\text{ask size} + \text{bid size}}} \times \text{best ask price}
$$


**Spread**:
- Difference between the best ask and bid prices.
$$\text{Spread} = \text{best ask price} - \text{best bid price}$$


**Rolling Spread Mean, Coefficient of Variation & Z-score**:
- Instead of just using the instantaneous spread, we track its mean, coefficient of variation and z-score over a given rolling time period.
$$
\text{Spread Mean}_t = \frac{1}{T} \sum_{i=t-T}^{t} \text{Spread}_i
$$
$$
\text{Spread CV}_t = \frac{\text{Spread Std}_t}{\text{Spread Mean}_t}
$$
$$
\text{Spread Z-score}_t = \frac{\text{Spread}_t - \text{Spread Mean}_t}{\text{Spread Std}_t}
$$

$$\frac{\text{Spread Std}_t} = \sqrt{\frac{1}{T} \sum_{i=t-T}^{t} (\text{Spread}_i - \text{Spread Mean}_t)^2}$$

- This helps detect **spread widening** events, spread instability (CV) and liquidity shocks.


**Trade direction (sign)**:
- we mark trades with +1 for buyer-initiated, -1 for seller-initiated.
    - Find the best bid and offer as of the trade time
    - Compare the trade price to them
    - If the trade price equals the best bid, mark as seller-initiated
    - If the trade price equals the best offer, mark as buyer-initiated
    - Otherwise, mark according to whether trade price was below or above mid price

**Size imbalance**:
- A common intuition among traders is that the order sizes displayed at the top of the book reflect the general intention of the market. When the number of shares at the bid exceeds that at the ask, participants  expect the next price movement to be upwards, and vice versa. 

$$\text{SI} = \frac{\text{ask size} - \text{bid size}}{\text{ask size} + \text{bid size}}$$


**Order imbalance**:
- The order imbalance is the absolute difference between the volume of buyer-initiated and seller-initiated trades, divided by their sum over a _fixed volume bucket_.
- This metric is designed to measure market stress, not directional bias, which is why we take the absolute value—positive and negative imbalances would cancel each other out, potentially understating the overall level of informed trading.

$$\text{I}_n = \frac{\left| V_n^B - V_n^S \right|}{V_n^B + V_n^S}$$

(_where $V_n^B$ and $V_n^S$ are the volume of buyer-initiated and seller-initiated trades in the _n-th fixed volume bucket_, respectively._)


**Volume-weighted probability of informed trading (VPIN)**:
- The rolling average of Order Imbalance over the last _$N$ volume buckets_.

$$VPIN = \frac{1}{N} \sum_{n=1}^{N} I_n$$

where each $OI_n$ corresponds to the order imbalance calculated for a fixed-volume bucket, ensuring that VPIN reflects market activity in a volume-synchronized manner rather than fixed time or trade intervals.


**Trade Flow**:
- An extension of the VPIN, a running tally of signed trade sizes where the sign is defined as 1 if the trade was seller-initiated and -1 if it was buyer-initiated.
- At any moment, we examine all reported trades within the last time period of length $\tau$.
- The essential idea behind flow as a quantitative metric is that, in circumstances when many sellers are willing to cross that market-making bid-offer spread to complete their transactions, there is likely to be new information driving their choices. We do not know exactly what it is, but we certainly want to adapt to it.

- When flow has a stable sign over macroscopic periods of time, it is a manifestation of supply and demand imbalance, and so it (usually) corresponds to steady price movements over a given time period.

$$F_t^{(\tau)} = V_{(t-\tau, t)}^B - V_{(t-\tau, t)}^S$$

- here we will use $\tau$ of 1 second and the time step $t$ of 100ms.


**Order Flow Imbalance (OFI)**:
- A measure of excess buying or selling pressure at the top of the order book, it represents the changes in supply and demand at the best bid and ask prices.
$$
\text{OFI} = (\text{Best Bid Size}_{t} - \text{Best Bid Size}_{t-1}) - (\text{Best Ask Size}_{t} - \text{Best Ask Size}_{t-1})
$$
- Best bid or size at the best bid increase -> increase in demand.
- Best bid or size at the best bid decreases -> decrease in demand.
- Best ask decreases or size at the best ask increases -> increase in supply.
- Best ask increases or size at the best ask decreases -> decrease in supply.


**Market Pressure (MP)**:
- Captures the aggressiveness of market orders relative to available liquidity.
- Higher values indicate stronger directional pressure.
$$
\text{MP}_t = \frac{\text{Trade Volume}_t}{\text{Bid Size}_t + \text{Ask Size}_t}


In [7]:
feats_df = extract_features(merged_trades,
                            half_life_s=60,
                            spread_window="60s",
                            flow_time_step="100ms",
                            flow_window="5s",
                            bucket_size=1000,
                            vpin_buckets=30,
                            vwap_past_window="60s",
                            vwap_future_window="20s")

In [8]:
print(feats_df.tail(10))

                             time_trade    price  mid_price  owa_price  ewma_price_return  ewma_mid_price_return  ewma_owa_price_return  past_vwap  future_vwap  trade_sign  duration_since_last_trade  size_imbalance  market_pressure    OFI  trade_flow  spread_mean  spread_CV  spread_Zscore
507684 2024-03-07 15:59:59.991157-05:00 514.8700   514.8750   514.8733            -0.0004                -0.0004                -0.0004   514.4033     514.8655          -1                     0.0000          0.6190           0.2400   -900 -95088.0000       0.0100     0.3134         0.0151
507685 2024-03-07 15:59:59.991361-05:00 514.8700   514.8750   514.8733            -0.0004                -0.0004                -0.0004   514.4034     514.8645          -1                     0.0002          0.6190           0.1905      0 -95088.0000       0.0100     0.3134         0.0151
507686 2024-03-07 15:59:59.991698-05:00 514.8700   514.8700   514.8697            -0.0004                -0.0004                -0

## 3. Analysis

Target variable:
- **Future VWAP**: Volume Weighted Average Price over the next x minutes of data, where x is 1/3 of the historical time interval using 

